In [1]:
import pandas as pd
import numpy as np
import sqlite3
import json
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
EXPERIMENT_NAME = 'P3856'
RUN_NAME = 'P3856_YHE211_1_Slot1-1_1_5104'

In [3]:
MZ_TOLERANCE_PPM = 5
MZ_TOLERANCE_PERCENT = MZ_TOLERANCE_PPM * 10**-4
SCAN_TOLERANCE = 5
RT_TOLERANCE = 5

##### 3DID parameters

##### PASEF parameters

In [4]:
# load the detected features
EXPERIMENT_DIR = '/data2/experiments/{}'.format(EXPERIMENT_NAME)
FEATURES_DIR = '{}/features'.format(EXPERIMENT_DIR)
FEATURES_FILE = '{}/detected-features-no-recal.sqlite'.format(FEATURES_DIR)
FEATURES_DEDUP_FILE = '{}/exp-{}-run-{}-features-pasef-dedup.pkl'.format(FEATURES_DIR, EXPERIMENT_NAME, RUN_NAME)
MZ_COLUMN_NAME = 'monoisotopic_mz'

In [5]:
RT_LOWER = 1650
RT_UPPER = 2200

db_conn = sqlite3.connect(FEATURES_FILE)
features_df = pd.read_sql_query("select * from features where rt_apex >= {} and rt_apex <= {} and run_name=='{}'".format(RT_LOWER, RT_UPPER, RUN_NAME), db_conn)
db_conn.close()

In [6]:
print('there are {} features prior to de-dup'.format(len(features_df)))

there are 252796 features prior to de-dup


In [7]:
features_df['dup_mz'] = features_df[MZ_COLUMN_NAME]  # shorthand to reduce verbosity
features_df['dup_mz_ppm_tolerance'] = features_df.dup_mz * MZ_TOLERANCE_PERCENT / 100
features_df['dup_mz_lower'] = features_df.dup_mz - features_df.dup_mz_ppm_tolerance
features_df['dup_mz_upper'] = features_df.dup_mz + features_df.dup_mz_ppm_tolerance
features_df['dup_scan_lower'] = features_df.scan_apex - SCAN_TOLERANCE
features_df['dup_scan_upper'] = features_df.scan_apex + SCAN_TOLERANCE
features_df['dup_rt_lower'] = features_df.rt_apex - RT_TOLERANCE
features_df['dup_rt_upper'] = features_df.rt_apex + RT_TOLERANCE
features_df['dup_composite_key'] = features_df.apply(lambda row: '{},{}'.format(row.feature_id, row.precursor_id), axis=1)

In [8]:
# remove these after we're finished
columns_to_drop_l = []
columns_to_drop_l.append('dup_mz')
columns_to_drop_l.append('dup_mz_ppm_tolerance')
columns_to_drop_l.append('dup_mz_lower')
columns_to_drop_l.append('dup_mz_upper')
columns_to_drop_l.append('dup_scan_lower')
columns_to_drop_l.append('dup_scan_upper')
columns_to_drop_l.append('dup_rt_lower')
columns_to_drop_l.append('dup_rt_upper')
columns_to_drop_l.append('dup_composite_key')


In [9]:
features_df.sort_values(by=['intensity'], ascending=False, inplace=True)

In [10]:
# see if any detections have a duplicate - if so, find the dup with the highest intensity and keep it
keep_l = []
for row in features_df.itertuples():
    dup_df = features_df[(features_df.dup_mz > row.dup_mz_lower) & (features_df.dup_mz < row.dup_mz_upper) & (features_df.scan_apex > row.dup_scan_lower) & (features_df.scan_apex < row.dup_scan_upper) & (features_df.rt_apex > row.dup_rt_lower) & (features_df.rt_apex < row.dup_rt_upper)].copy()
    # group the dups by charge - take the most intense for each charge
    for group_name,group_df in dup_df.groupby(['charge'], as_index=False):
        keep_l.append(group_df.iloc[0].dup_composite_key)


In [11]:
# remove any features that are not in the keep list
dedup_df = features_df[features_df.dup_composite_key.isin(keep_l)].copy()

In [12]:
number_of_dups = len(features_df)-len(dedup_df)
print('removed {} duplicates ({}% of the original detections)'.format(number_of_dups, round(number_of_dups/len(features_df)*100)))
print('there are {} detected de-duplicated features'.format(len(dedup_df)))

removed 28830 duplicates (11% of the original detections)
there are 223966 detected de-duplicated features


In [13]:
# remove the columns we added earlier
dedup_df.drop(columns_to_drop_l, axis=1, inplace=True)

In [14]:
dedup_df.sample(n=3)

,monoisotopic_mz,charge,intensity,intensity_full_rt_extent,intensity_six_sigma,scan_apex,scan_lower,scan_upper,rt_apex,rt_lower,rt_upper,precursor_id,envelope,deconvolution_score,envelope_mono_peak_intensity,feature_id,monoisotopic_mass,run_name
42486,592.548165,4,12409.000000,363,335,847.72,836.26,859.19,1969.84,1962.93,1976.75,29215,"[[592.8017395419953, 3448.0], [592.80173954199...",70.609796,3448.0,2921503,2366.163461,P3856_YHE211_1_Slot1-1_1_5104
50675,927.990951,2,85381.470588,89190,56884,267.86,247.86,287.86,2016.79,2014.83,2018.74,32570,"[[927.9837258289976, 24392.470588235294], [928...",361.989940,23656.0,3257001,1853.967303,P3856_YHE211_1_Slot1-1_1_5104
108131,1001.951778,5,34858.000000,1074,1074,544.68,524.68,564.68,2052.22,2042.22,2062.22,34572,"[[1001.9604635907656, 124.0], [1002.1518782997...",68.441413,124.0,3457204,5004.722392,P3856_YHE211_1_Slot1-1_1_5104


In [15]:
dedup_df.to_pickle(FEATURES_DEDUP_FILE)